# EDA: Market

__Notes__ code scripts at the bottom. Up the set to 2M with all assets. Set test set as 2017. 

In [2]:
# !pip freeze


In [3]:
import sys
paths = ['../scripts',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union']

for path in paths:
    sys.path.append(path)

from market_imports import *
from market_code import *
from market_trans import *
from quickpipe_mod import * 
from pandas_feature_union import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')


import warnings
warnings.filterwarnings(action='ignore')

import datetime as dt



## Data Prep

In [4]:
# df_market.head()



In [5]:
# TRYING TO PREVENT DATE TYPE CONVERSION
# THIS SHOULD BE UPDATED FROM PANDA DATAREADER
csv = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

csv.shape

(57384, 9)

In [6]:
# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returnsOpenNextMktres10'] = result.resid
# Lose the first and last day
csv = csv[-csv['returnsOpenNextMktres10'].isnull()]
csv.drop(['returns_open_raw10_next'], axis=1, inplace=True)

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']

# Matching the kernel data some more
csv.index.name = 'time'
csv.reset_index(inplace=True)
csv.rename(columns={'asset': 'assetCode', 'Volume': 'volume',
                  'Close': 'close'}, inplace=True)
csv.head()

,time,assetCode,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returnsOpenNextMktres10
0,2014-01-16,QCOM,74.470001,74.720001,10263200.0,0.002814,0.013519,0.018914,0.011615,-0.015252
1,2014-01-16,KEP,16.660000,16.719999,205300.0,0.009012,0.000000,0.017497,0.009650,-0.010187
2,2014-01-16,NWS,17.059999,17.040001,506000.0,-0.006435,-0.011076,-0.048678,-0.044707,0.050676
3,2014-01-16,BHP,66.570000,66.889999,5735900.0,0.029432,0.033296,-0.001494,-0.012391,0.013469
4,2014-01-16,NGG,70.043671,69.934494,212800.0,0.001875,0.004530,-0.004361,-0.009308,-0.042414


In [7]:
csv.shape

(35818, 10)

## Feature Engineer  

Ideas: momentum, Beta, indicators

In [6]:
# data.query("asset == 'AAPL'")

In [7]:
# test.query("asset == 'AAPL'")[['Close']].diff().head()

vectorize  

https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm

In [18]:
df = csv.copy()

# SEPARATE TEST DATA EARLY TO SIMULATE KERNEL
# This is a bit silly. Just create a test matrix for 2017. It is
# exactely what 2 sigma is doing anyway


# TEMPORARILY INTRODUCE AN INTEGER DATE FEATURE
df['date_'] = df.time.dt.strftime('%Y%m%d').astype(int)
dates = df['date_'].unique().tolist()
print('{} dates \nreduced to {} for testing'.format( len(dates),                                                     int(len(dates)*.90) )) 

base = df[ df['date_'].isin( dates[ :int(len(dates)*.90)] ) ]
base.drop(['date_'], axis=1, inplace=True)
base.reset_index(drop=True, inplace=True)
print('\nbase: ', base.shape, '\nindex reset')


# TEST DATA FOR ENV SIMULATION
data_test = df[ df['date_'].isin( dates[ int(len(dates)*.90):] ) ]
test_dates = data_test['date_'].unique().tolist()
# the incoming test data will not have a response
data_test.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
# data_test.drop(['date_'], axis=1, inplace=True)
print('\ntest: ', data_test.shape)


# SETTING UP THE TEST TRIAL
data_test.set_index(['date_', 'assetCode'], inplace=True)
data_test.sort_index(inplace=True)
print('\ntest data is set up as a sorted multi-index. This allows \nit \
to be used as an interator')

758 dates 
reduced to 682 for testing

base:  (32225, 10) 
index reset

test:  (3593, 10)

test data is set up as a sorted multi-index. This allows 
it to be used as an interator


__Notes:__ I tested the groupby transform against an subset using only ```.rolling()```. The results were the same. I also tested ```.rolling()``` with a time vs int index - no difference. I tried to direct ```.rolling()``` to use the time col ```on = col```, this threw an error.   

__Fairly certain the ```.rolling()``` method is indifferent as to whether the index is a time feature. It probably only requires a strictly increasing integer index__

In [27]:
# IF YOU WANT TO UPDATE

# with open('/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/output/asset_full.txt', 'r') as file:
#     assets = file.read()
# file.close()  
##################################

# assets = ['CHL.N', 'RAD.N', 'HBC', 'PDNT.O', 'IGLD.O', '0857.F', 'GOOGa.DE', 'DIS.N', '1285.HK', 'DIS.F', 'MTTX.O', 'NDAQ.O', 'ALBK.F', 'AAPL.OQ', 'GS.N', 'WF.N', 'HTX.N', 'UPM1V.HE', 'GE.N', 'LPL.N', 'RYAAY.OQ', 'MHR.A', 'KB.N', '1301.HK', 'YHOO.O', 'SRIB.O', 'RYAAY.O', '2332.DE', 'YHOO.OQ', 'TRV.DE', '0941.HK', 'SIRI.O', 'UPM.N', 'SIEB.O', 'DWMT.O', 'NGG.N', '3777.HK', 'NML.A', 'NWS.N', 'SAN.MC', 'PGEB.O', 'MER.N', 'DSP.A', 'MON.DE', 'NOKy.DE', 'ADBE.DE', 'MTTT.O', 'SI.N', '3838.HK', 'NG.L', 'NOKS.DE', 'QCOM.DE', 'DIS.DE', 'CS.AS', 'TSM.N', 'C', 'MTNK.O', 'AAPL.F', '0005.HK', 'QCOM.O', 'IGLD.TA', 'ADBE.OQ', 'SIEB.OQ', 'STZb.N', 'AIB.N', 'BEN.N', 'IGLD.OQ', 'SNE.N', 'TWX.N', 'DCX.N', 'THC.N', 'MSFT.DE', 'NWSa.N', 'ISAT.JK', '0857.HK', '053000.KS', 'NWS.AX', 'WMT.DE', 'AUO.N', 'SIEGnq.L', 'KEP.N', 'BHP.F', 'ELC.A', '2332.HK', 'AAPL.DE', 'NOK1V.HE', 'WSH.N', '2330.TW', 'MX.N', 'NOKS.F', 'XMSR.O', 'FMSAUBD.MX', 'BHP.N', 'SIRI.OQ', 'PLJC.O', 'MSPX.O', 'NOK.N', 'YHOO.DE', 'CHLy.DE', 'DCXGn.F', 'SIEGn.DE', 'STZ.N', '0941.F', 'STD.BA', 'ALBK.I', '015760.KS', 'STD.N', 'VIAb.N', 'CSNA3.SA', '7267.T', 'MSFT.O', 'C.N', 'AAPL.O', '0941.DE', 'MNNY.O', 'RBS.L', 'TIE.N', 'HMC.N', '6758.T', 'HSBA.L', 'NDAQ.OQ', 'ALBK.L', 'MTSM.O', 'EST.A', 'IIT.N', 'WMT.N', '0857.DE', 'VIA.N', 'RBV.N', 'MLN.P', 'BHP.AX', 'CTCH.O', 'RBS.DE', 'CGA.N', 'MON.F', 'CS.L', 'GOOG.OQ', 'MON.N', 'SASY.PA', 'MEO1V.HE', 'SIEGn.F', '2409.TW', 'DCXGn.DE', 'STA.N', 'SASY.F', 'ADBE.O', 'SNY.N', 'RBS', 'RYA.I', '034220.KS', 'RYA.L', 'MSFT.OQ', 'RYA.F', 'MSFT.F', 'HBC.N', 'VNV.N', 'HBC.PA', 'FMX.N', 'QCOM.OQ', '060000.KS', 'TRV.F', 'GOOG.O', 'PTR.N', 'SID.N']
# sub_asst = assets[:200]

# sub = [i.split('.')[0] for i in sub_asst]     

# data = marketDataGen(sub)

# data.to_csv(data_path + '/sigma_data.csv')

In [41]:
# assets = ['A.N', 'AAI.N', 'AAP.N', 'AAPL.O', 'ABB.N', 'ABC.N', 'ABD.N', 'ABM.N', 'ABT.N', 'ABV.N', 'ABY.N', 'ACF.N', 'ACH.N', 'ACL.N', 'ACS.N', 'ACTS.O', 'ACXM.O', 'AD.N', 'ADBE.O', 'ADCT.O', 'ADI.N', 'ADM.N', 'ADP.N', 'ADS.N', 'ADSK.O', 'ADTN.O', 'AEA.N', 'AEE.N', 'AEG.N', 'AEIS.O', 'AEL.N', 'AEOS.O', 'AEP.N', 'AER.N', 'AES.N', 'AET.N', 'AF.N', 'AFFX.O', 'AFG.N', 'AFL.N', 'AFR.N', 'AG.N', 'AGE.N', 'AGN.N', 'AGU.N', 'AH.N', 'AHG.N', 'AHL.N', 'AHS.N', 'AHT.N', 'AIN.N', 'AINV.O', 'AIR.N', 'AIT.N', 'AIV.N', 'AIZ.N', 'AJG.N', 'AKAM.O', 'AKR.N', 'AKS.N', 'ALB.N', 'ALD.N', 'ALGN.O', 'ALK.N', 'ALL.N', 'ALNY.O', 'ALO.N', 'ALTR.O', 'ALU.N', 'ALV.N', 'ALXN.O', 'AMAT.O', 'AMD.N', 'AME.N', 'AMED.O', 'AMG.N', 'AMGN.O', 'AMKR.O', 'AMLN.O', 'AMMD.O', 'AMP.N', 'AMT.N', 'AMTD.O', 'AMX.N', 'AMZN.O', 'AN.N', 'ANDE.O', 'ANDW.O', 'ANF.N', 'ANR.N', 'AOC.N', 'AOS.N', 'APA.N', 'APC.N', 'APD.N', 'APH.N', 'APKT.O', 'APOL.O', 'APPB.O', 'AQNT.O', 'ARCC.O', 'ARE.N', 'ARG.N', 'ARW.N', 'ARXT.O', 'ARXX.O', 'ASBC.O', 'ASCA.O', 'ASH.N', 'ASN.N', 'AT.N', 'ATHR.O', 'ATI.N', 'ATMI.O', 'ATML.O', 'ATO.N', 'ATPG.O', 'ATRS.O', 'ATU.N', 'ATW.N', 'AU.N', 'AUO.N', 'AUY.N', 'AV.N', 'AVA.N', 'AVB.N', 'AVCT.O', 'AVP.N', 'AVT.N', 'AVX.N', 'AVY.N', 'AW.N', 'AXE.N', 'AXL.N', 'AXP.N', 'AXR.N', 'AXS.N', 'AYE.N', 'AYI.N', 'AZN.N', 'AZO.N', 'B.N', 'BA.N', 'BAC.N', 'BAX.N', 'BBBY.O', 'BBD.N', 'BBT.N', 'BBV.N', 'BBY.N', 'BC.N', 'BCE.N', 'BCO.N', 'BCR.N', 'BCS.N', 'BDC.N', 'BDK.N', 'BDN.N', 'BDX.N', 'BEAS.O', 'BEAV.O', 'BEC.N', 'BECN.O', 'BEE.N', 'BEN.N', 'BER.N', 'BEZ.N', 'BFb.N', 'BG.N', 'BGC.N', 'BGG.N', 'BGP.N', 'BHE.N', 'BHI.N', 'BHP.N', 'BHS.N', 'BID.N', 'BIDU.O', 'BIG.N', 'BIIB.O', 'BJ.N', 'BJS.N', 'BK.N', 'BKC.N', 'BKD.N', 'BKS.N', 'BLC.N', 'BLG.N', 'BLKB.O', 'BLL.N', 'BLUD.O', 'BMA.N', 'BMC.N', 'BMET.O', 'BMR.N', 'BMRN.O', 'BMS.N', 'BMY.N', 'BNI.N', 'BNK.N', 'BOBJ.O', 'BOH.N', 'BOL.N', 'BOT.N', 'BOW.N', 'BP.N', 'BQI.A', 'BRC.N', 'BRCM.O', 'BRE.N', 'BRKL.O', 'BRKS.O', 'BRL.N', 'BRO.N', 'BRS.N', 'BRY.N', 'BSC.N', 'BSG.N', 'BSX.N', 'BT.N', 'BUCY.O', 'BUD.N', 'BVN.N', 'BWA.N', 'BWP.N', 'BWS.N', 'BXP.N', 'BYD.N', 'CA.N', 'CAB.N', 'CAG.N', 'CAH.N', 'CAI.N', 'CAJ.N', 'CAL.N', 'CAM.N', 'CAO.N', 'CAR.N', 'CAT.N', 'CB.N', 'CBH.N', 'CBI.N', 'CBK.N', 'CBL.N', 'CBS.N', 'CBSS.O', 'CBST.O', 'CBT.N', 'CC.N', 'CCJ.N', 'CCK.N', 'CCL.N', 'CCMP.O', 'CCOI.O', 'CCU.N', 'CDNS.O', 'CDR.N', 'CDWC.O', 'CE.N', 'CECO.O', 'CEG.N', 'CEI.N', 'CELG.O', 'CEN.N', 'CENT.O', 'CENX.O', 'CEO.N', 'CEPH.O', 'CERN.O', 'CF.N', 'CFC.N', 'CFR.N', 'CG.N', 'CHAP.O', 'CHD.N', 'CHE.N', 'CHH.N', 'CHIC.O', 'CHK.N', 'CHL.N', 'CHRW.O', 'CHS.N', 'CHU.N', 'CI.N', 'CIB.N', 'CIEN.O', 'CIG.N', 'CINF.O', 'CKFR.O', 'CKH.N', 'CKR.N', 'CL.N', 'CLB.N', 'CLC.N', 'CLE.N', 'CLI.N', 'CLP.N', 'CLS.N', 'CLX.N', 'CMA.N', 'CMC.N', 'CMCSA.O', 'CMCSK.O', 'CMED.O', 'CMG.N', 'CMGb.N', 'CMI.N', 'CMP.N', 'CMS.N', 'CMVT.OQ', 'CMX.N', 'CNA.N', 'CNC.N', 'CNE.N', 'CNET.O', 'CNI.N', 'CNL.N', 'CNP.N', 'CNQ.N', 'CNS.N', 'CNW.N', 'COCO.O', 'COF.N', 'COG.N', 'COGN.O', 'COGT.O', 'COH.N', 'COL.N', 'COLM.O', 'COMS.O', 'COO.N', 'COP.N', 'COST.O', 'CP.N', 'CPA.N', 'CPB.N', 'CPS.N', 'CPT.N', 'CPWR.O', 'CPX.N', 'CQB.N', 'CR.N', 'CRA.N', 'CRDN.O', 'CREE.O', 'CRI.N', 'CRL.N', 'CRM.N', 'CROX.O', 'CRR.N', 'CRS.N', 'CRZO.O', 'CS.N', 'CSCO.O', 'CSE.N', 'CSGS.O', 'CSH.N', 'CSX.N', 'CTAS.O', 'CTB.N', 'CTCM.O', 'CTL.N', 'CTRP.O', 'CTSH.O', 'CTV.N', 'CTX.N', 'CTXS.O', 'CUZ.N', 'CVA.N', 'CVC.N', 'CVD.N', 'CVG.N', 'CVH.N', 'CVS.N', 'CVTX.O', 'CVX.N', 'CX.N', 'CXG.N', 'CY.N', 'CYH.N', 'CYMI.O', 'CYN.N', 'CYT.N', 'CYTC.O', 'D.N', 'DADE.O', 'DAKT.O', 'DBD.N', 'DCEL.O', 'DCI.N', 'DD.N', 'DDS.N', 'DE.N', 'DEI.N', 'DELL.O', 'DEO.N', 'DG.N', 'DGX.N', 'DHI.N', 'DHR.N', 'DIGE.O', 'DIOD.O', 'DIS.N', 'DIVX.O', 'DJ.N', 'DKS.N', 'DLB.N', 'DLM.N', 'DLP.N', 'DLR.N', 'DLX.N', 'DNA.N', 'DNB.N', 'DNR.N', 'DO.N', 'DOV.N', 'DOW.N', 'DOX.N', 'DPL.N', 'DPZ.N', 'DQE.N', 'DRC.N', 'DRE.N', 'DRH.N', 'DRI.N', 'DRIV.O', 'DRQ.N', 'DST.N', 'DSW.N', 'DTE.N', 'DTG.N', 'DV.N', 'DVA.N', 'DVN.N', 'DWA.N', 'DY.N', 'E.N', 'EAC.N', 'EAGL.O', 'EAS.N', 'EAT.N', 'EBAY.O', 'ECA.N', 'ECL.N', 'ECLP.O', 'ED.N', 'EDS.N', 'EDU.N', 'EE.N', 'EEFT.O', 'EFD.N', 'EFII.O', 'EFX.N', 'EGN.N', 'EIX.N', 'EL.N', 'ELK.N', 'ELN.N', 'ELOS.O', 'ELP.N', 'ELX.N', 'ELY.N', 'EMC.N', 'EME.N', 'EMN.N', 'EMR.N', 'ENH.N', 'ENN.N', 'ENTG.O', 'EOG.N', 'EP.N', 'EPD.N', 'EPIC.O', 'EQ.N', 'EQIX.O', 'EQR.N', 'EQY.N', 'ERIC.O', 'ERTS.O', 'ESE.N', 'ESV.N', 'ETH.N', 'ETM.N', 'ETP.N', 'ETR.N', 'EV.N', 'EW.N', 'EWBC.O', 'EXC.N', 'EXEL.O', 'EXP.N', 'EXPD.O', 'EXR.N', 'EYE.N', 'F.N', 'FAST.O', 'FCH.N', 'FCL.N', 'FCN.N', 'FCS.N', 'FCX.N', 'FDC.N', 'FDO.N', 'FDRY.O', 'FDS.N', 'FDX.N', 'FE.N', 'FEIC.O', 'FFIV.O', 'FHN.N', 'FIC.N', 'FII.N', 'FIS.N', 'FISV.O', 'FITB.O', 'FL.N', 'FLEX.O', 'FLIR.O', 'FLML.O', 'FLO.N', 'FLR.N', 'FLS.N', 'FMC.N', 'FMCN.O', 'FMER.O', 'FMX.N', 'FNB.N', 'FNFG.O', 'FOE.N', 'FORM.O', 'FR.N', 'FRK.N', 'FRT.N', 'FRX.N', 'FSLR.O', 'FTI.N', 'FTO.N', 'FUL.N', 'FWRD.O', 'GAP.N', 'GAS.N', 'GBBK.O', 'GCO.N', 'GD.N', 'GDI.N', 'GDP.N', 'GE.N', 'GENZ.O', 'GFI.N', 'GG.N', 'GGB.N', 'GGG.N', 'GGL.N', 'GI.N', 'GILD.O', 'GIS.N', 'GLS.N', 'GLT.N', 'GLW.N', 'GME.N', 'GMR.N', 'GMST.O', 'GMT.N', 'GNA.N', 'GNTX.O', 'GNW.N', 'GOLD.O', 'GPC.N', 'GPI.N', 'GPN.N', 'GPS.N', 'GR.N', 'GRA.N', 'GRP.N', 'GS.N', 'GSIC.O', 'GSK.N', 'GSS.A', 'GT.N', 'GTI.N', 'GTRC.O', 'GTW.N', 'GVA.N', 'GW.A', 'GWR.N', 'GWW.N', 'GXP.N', 'GYI.N', 'GYMB.O', 'H.N', 'HAL.N', 'HAR.N', 'HAS.N', 'HBAN.O', 'HBC.N', 'HBI.N', 'HC.N', 'HCBK.O', 'HCC.N', 'HCR.N', 'HD.N', 'HES.N', 'HET.N', 'HEW.N', 'HGSI.O', 'HIG.N', 'HIH.N', 'HIMX.O', 'HIW.N', 'HL.N', 'HLF.N', 'HLT.N', 'HLX.N', 'HLYS.O', 'HMA.N', 'HMC.N', 'HME.N', 'HMN.N', 'HMY.N', 'HNT.N', 'HNZ.N', 'HOG.N', 'HOLX.O', 'HON.N', 'HOS.N', 'HOT.N', 'HOV.N', 'HP.N', 'HPC.N', 'HPT.N', 'HR.N', 'HRB.N', 'HRL.N', 'HRS.N', 'HSC.N', 'HSIC.O', 'HSP.N', 'HST.N', 'HSY.N', 'HTLD.O', 'HTZ.N', 'HUBG.O', 'HUBb.N', 'HUM.N', 'HUN.N', 'HW.N', 'HWAY.O', 'HXL.N', 'HXM.N', 'HYDL.O', 'HYSL.O', 'IAG.N', 'IBM.N', 'IBN.N', 'ICO.N', 'ICON.O', 'ID.N', 'IDA.N', 'IDTI.O', 'IFF.N', 'IFIN.O', 'IFS.N', 'IKN.N', 'ILA.N', 'ILMN.O', 'IM.N', 'IMCL.O', 'IN.N', 'INFA.O', 'INFY.O', 'ING.N', 'INT.N', 'INTC.O', 'INTU.O', 'INWK.O', 'IP.N', 'IPCR.O', 'IPG.N', 'IPS.N', 'IRC.N', 'IRF.N', 'ISE.N', 'ISIL.O', 'ISIS.O', 'ISLE.O', 'ISRG.O', 'IT.N', 'ITG.N', 'ITMN.O', 'ITRI.O', 'ITW.N', 'IWA.N', 'IWOV.O', 'JAH.N', 'JBHT.O', 'JBL.N', 'JBLU.O', 'JBX.N', 'JCG.N', 'JCI.N', 'JCP.N', 'JEC.N', 'JEF.N', 'JH.N', 'JKHY.O', 'JLL.N', 'JNC.N', 'JNJ.N', 'JNPR.O', 'JNS.N', 'JOE.N', 'JPM.N', 'JWN.N', 'K.N', 'KAR.N', 'KBH.N', 'KBR.N', 'KCI.N', 'KDN.N', 'KEA.N', 'KEP.N', 'KEX.N', 'KEY.N', 'KFN.N', 'KFY.N', 'KG.N', 'KGC.N', 'KIM.N', 'KKD.N', 'KLAC.O', 'KMB.N', 'KMI.N', 'KMP.N', 'KMT.N', 'KMX.N', 'KND.N', 'KNX.N', 'KO.N', 'KOMG.O', 'KPA.N', 'KR.N', 'KRC.N', 'KRY.A', 'KSE.N', 'KSS.N', 'KSU.N', 'KTC.N', 'KWD.N', 'KWK.N', 'KYPH.O', 'LAUR.O', 'LAZ.N', 'LCAV.O', 'LCC.N', 'LDG.N', 'LEA.N', 'LEAP.O', 'LEE.N', 'LEG.N', 'LEN.N', 'LEND.O', 'LFB.N', 'LFC.N', 'LGF.N', 'LH.N', 'LHO.N', 'LI.N', 'LIFC.O', 'LII.N', 'LKQX.O', 'LLTC.O', 'LLY.N', 'LM.N', 'LMT.N', 'LNC.N', 'LNCR.O', 'LNT.N', 'LOW.N', 'LPL.N', 'LPNT.O', 'LPX.N', 'LRCX.O', 'LRY.N', 'LSCC.O', 'LSI.N', 'LSS.N', 'LSTR.O', 'LTM.N', 'LTR.N', 'LUK.N', 'LUV.N', 'LVS.N', 'LWSN.O', 'LXK.N', 'LXP.N', 'LYO.N', 'LYV.N', 'LZ.N', 'MA.N', 'MAC.N', 'MAN.N', 'MAR.N', 'MAS.N', 'MAT.N', 'MATK.O', 'MBI.N', 'MBT.N', 'MCCC.O', 'MCD.N', 'MCGC.O', 'MCHP.O', 'MCK.N', 'MCO.N', 'MCRL.O', 'MDC.N', 'MDCO.O', 'MDP.N', 'MDRX.O', 'MDU.N', 'ME.N', 'MEDI.O', 'MEDX.O', 'MEE.N', 'MEL.N', 'MENT.O', 'MER.N', 'MET.N', 'MFC.N', 'MFE.N', 'MGA.N', 'MGLN.O', 'MGM.N', 'MHK.N', 'MHS.N', 'MIL.N', 'MIR.N', 'MKC.N', 'MKSI.O', 'MLHR.O', 'MLI.N', 'MLM.N', 'MLNM.O', 'MLS.N', 'MMC.N', 'MMM.N', 'MMS.N', 'MO.N', 'MOGN.O', 'MOLX.O', 'MON.N', 'MPG.N', 'MPS.N', 'MR.N', 'MRBK.O', 'MRD.N', 'MRH.N', 'MRO.N', 'MRVL.O', 'MRX.N', 'MS.N', 'MSCC.O', 'MSFT.O', 'MSM.N', 'MSTR.O', 'MTB.N', 'MTD.N', 'MTG.N', 'MTH.N', 'MTN.N', 'MTZ.N', 'MU.N', 'MUR.N', 'MVL.N', 'MW.N', 'MWAb.N', 'MWV.N', 'MXIM.O', 'MYGN.O', 'MYL.N', 'NAL.N', 'NAV.N', 'NBL.N', 'NBR.N', 'NCC.N', 'NCI.N', 'NCR.N', 'NCX.N', 'NDAQ.O', 'NDN.N', 'NEM.N', 'NFG.N', 'NFLX.O', 'NFP.N', 'NFS.N', 'NFX.N', 'NG.A', 'NHP.N', 'NI.N', 'NITE.O', 'NJR.N', 'NKE.N', 'NKTR.O', 'NLC.N', 'NLY.N', 'NMR.N', 'NMX.N', 'NNI.N', 'NNN.N', 'NOC.N', 'NOK.N', 'NOV.N', 'NOVL.O', 'NRG.N', 'NRPH.O', 'NSC.N', 'NSIT.O', 'NSM.N', 'NSR.N', 'NST.N', 'NTES.O', 'NTGR.O', 'NTRI.O', 'NTRS.O', 'NTY.N', 'NUAN.O', 'NUE.N', 'NVDA.O', 'NVL.N', 'NVS.N', 'NVT.N', 'NWL.N', 'NWY.N', 'NX.N', 'NXG.A', 'NXL.N', 'NXY.N', 'NYB.N', 'NYT.N', 'O.N', 'OCAS.O', 'OCN.N', 'OCR.N', 'ODFL.O', 'ODP.N', 'OFC.N', 'OGE.N', 'OHI.N', 'OI.N', 'OII.N', 'OIS.N', 'OKE.N', 'OLN.N', 'OMAB.O', 'OMC.N', 'OMG.N', 'OMI.N', 'OMM.N', 'OMTR.O', 'ONB.N', 'ONNN.O', 'OO.N', 'OPSW.O', 'ORB.N', 'ORCL.O', 'ORH.N', 'ORI.N', 'OSI.N', 'OSIP.O', 'OSK.N', 'OTEX.O', 'OVTI.O', 'OXPS.O', 'OXY.N', 'PAAS.O', 'PACR.O', 'PALM.O', 'PAS.N', 'PAY.N', 'PAYX.O', 'PBG.N', 'PBI.N', 'PBR.N', 'PBRa.N', 'PBY.N', 'PCAR.O', 'PCG.N', 'PCH.N', 'PCL.N', 'PCLN.O', 'PCP.N', 'PCU.N', 'PD.N', 'PDCO.O', 'PDE.N', 'PDLI.O', 'PEG.N', 'PEI.N', 'PENN.O', 'PEP.N', 'PER.N', 'PETM.O', 'PFCB.O', 'PFE.N', 'PFG.N', 'PG.N', 'PGI.N', 'PGL.N', 'PGR.N', 'PH.N', 'PHG.N', 'PHH.N', 'PHM.N', 'PHRM.O', 'PII.N', 'PKG.N', 'PKI.N', 'PKX.N', 'PL.N', 'PLAB.O', 'PLCE.O', 'PLCM.O', 'PLL.N', 'PLLL.O', 'PLT.N', 'PLXS.O', 'PMCS.O', 'PMI.N', 'PMTI.O', 'PNC.N', 'PNK.N', 'PNM.N', 'PNRA.O', 'PNW.N', 'PNY.N', 'POL.N', 'POM.N', 'POOL.O', 'POS.N', 'POT.N', 'PPDI.O', 'PPG.N', 'PPL.N', 'PPP.N', 'PPS.N', 'PRE.N', 'PRU.N', 'PRX.N', 'PSA.N', 'PSD.N', 'PSSI.O', 'PSUN.O', 'PSYS.O', 'PTEN.O', 'PTR.N', 'PTRY.O', 'PTV.N', 'PWR.N', 'PX.N', 'PXD.N', 'PXP.N', 'PYX.N', 'PZZA.O', 'Q.N', 'QCOM.O', 'QLGC.O', 'QSFT.O', 'R.N', 'RAD.N', 'RAI.N', 'RBC.N', 'RBN.N', 'RCII.O', 'RCL.N', 'RDA.N', 'RDC.N', 'RDN.N', 'RDSa.N', 'RDSb.N', 'RDY.N', 'RE.N', 'REG.N', 'REGN.O', 'RES.N', 'RF.N', 'RFMD.O', 'RGC.N', 'RGLD.O', 'RGS.N', 'RHI.N', 'RHT.N', 'RJF.N', 'RKT.N', 'RMBS.O', 'RMD.N', 'RNR.N', 'ROH.N', 'ROK.N', 'ROP.N', 'ROST.O', 'RPM.N', 'RRC.N', 'RS.N', 'RSG.N', 'RSH.N', 'RTI.N', 'RTN.N', 'RTP.N', 'RVBD.O', 'RVI.N', 'RX.N', 'RYI.N', 'RYL.N', 'RYN.N', 'SAF.N', 'SAFM.O', 'SAH.N', 'SAP.N', 'SAPE.O', 'SAY.N', 'SBAC.O', 'SBH.N', 'SBIB.O', 'SBS.N', 'SBUX.O', 'SCG.N', 'SCHL.O', 'SCHN.O', 'SCHW.O', 'SCI.N', 'SCRX.O', 'SCS.N', 'SCSS.O', 'SE.N', 'SEE.N', 'SEIC.O', 'SEPR.O', 'SFC.N', 'SFD.N', 'SFI.N', 'SFL.N', 'SGMS.O', 'SHLD.O', 'SHO.N', 'SHOO.O', 'SHPGY.O', 'SHW.N', 'SIAL.O', 'SID.N', 'SIE.N', 'SII.N', 'SIL.A', 'SIMG.O', 'SINA.O', 'SIRF.O', 'SIVB.O', 'SJM.N', 'SKIL.O', 'SKM.N', 'SKS.N', 'SKX.N', 'SKYF.O', 'SKYW.O', 'SLAB.O', 'SLB.N', 'SLG.N', 'SLM.N', 'SLR.N', 'SMG.N', 'SMI.N', 'SMOD.O', 'SMTC.O', 'SNA.N', 'SNDA.O', 'SNDK.O', 'SNE.N', 'SNH.N', 'SNP.N', 'SNPS.O', 'SNY.N', 'SO.N', 'SOHU.O', 'SON.N', 'SONC.O', 'SOV.N', 'SPG.N', 'SPIL.O', 'SPLS.O', 'SPN.N', 'SPR.N', 'SRE.N', 'SRX.N', 'SRZ.N', 'SSD.N', 'SSI.N', 'SSL.N', 'STEI.O', 'STI.N', 'STJ.N', 'STLD.O', 'STM.N', 'STN.N', 'STP.N', 'STR.N', 'STT.N', 'STTS.O', 'STZ.N', 'SUG.N', 'SUN.N', 'SVM.N', 'SVVS.O', 'SWC.N', 'SWFT.O', 'SWKS.O', 'SWN.N', 'SWS.N', 'SWX.N', 'SWY.N', 'SY.N', 'SYK.N', 'SYKE.O', 'SYMC.O', 'SYNA.O', 'SYY.N', 'T.N', 'TAM.N', 'TAP.N', 'TBL.N', 'TCB.N', 'TCO.N', 'TDW.N', 'TE.N', 'TECD.O', 'TEF.N', 'TEK.N', 'TEN.N', 'TEO.N', 'TER.N', 'TEVA.O', 'TEX.N', 'TG.N', 'TGT.N', 'THE.N', 'THG.N', 'THO.N', 'THOR.O', 'THX.N', 'TIBX.O', 'TIE.N', 'TIF.N', 'TIN.N', 'TJX.N', 'TK.N', 'TKC.N', 'TKLC.O', 'TKR.N', 'TLAB.O', 'TLB.N', 'TLK.N', 'TLM.N', 'TM.N', 'TMK.N', 'TMO.N', 'TMX.N', 'TNB.N', 'TNE.N', 'TNOX.O', 'TOL.N', 'TOT.N', 'TPX.N', 'TQNT.O', 'TRA.N', 'TRB.N', 'TRID.O', 'TRMB.O', 'TRN.N', 'TROW.O', 'TRP.N', 'TRW.N', 'TRYb.N', 'TS.N', 'TSCO.O', 'TSG.N', 'TSM.N', 'TSN.N', 'TSO.N', 'TSU.N', 'TSY.N', 'TTC.N', 'TTEK.O', 'TTI.N', 'TTM.N', 'TTWO.O', 'TUES.O', 'TUP.N', 'TV.N', 'TWB.N', 'TX.N', 'TXI.N', 'TXN.N', 'TXRH.O', 'TXT.N', 'TXU.N', 'TZIX.O', 'UAPH.O', 'UB.N', 'UBB.N', 'UCO.N', 'UEPS.O', 'UGI.N', 'UHS.N', 'UL.N', 'UN.N', 'UNFI.O', 'UNH.N', 'UNM.N', 'UNP.N', 'UNT.N', 'UPL.A', 'UPS.N', 'URBN.O', 'URI.N', 'URS.N', 'USB.N', 'USG.N', 'USPI.O', 'UST.N', 'UTHR.O', 'UTI.N', 'UTIW.O', 'UTX.N', 'UVN.N', 'VAL.N', 'VAR.N', 'VCI.N', 'VFC.N', 'VG.N', 'VIAb.N', 'VLO.N', 'VLY.N', 'VMC.N', 'VNO.N', 'VOL.N', 'VPHM.O', 'VRGY.O', 'VRSN.O', 'VRTX.O', 'VRX.N', 'VSEA.O', 'VSH.N', 'VTR.N', 'VVC.N', 'VZ.N', 'WAB.N', 'WAG.N', 'WAT.N', 'WB.N', 'WBS.N', 'WBSN.O', 'WCC.N', 'WCG.N', 'WCN.N', 'WDC.N', 'WDR.N', 'WEBX.O', 'WEN.N', 'WERN.O', 'WFC.N', 'WFSL.O', 'WGL.N', 'WGO.N', 'WHR.N', 'WIT.N', 'WL.N', 'WLK.N', 'WM.N', 'WMB.N', 'WMG.N', 'WMI.N', 'WMS.N', 'WMT.N', 'WNR.N', 'WOOF.O', 'WOR.N', 'WR.N', 'WRE.N', 'WRI.N', 'WSM.N', 'WSO.N', 'WST.N', 'WTI.N', 'WTR.N', 'WTS.N', 'WTW.N', 'WU.N', 'WW.N', 'WWW.N', 'WWY.N', 'WY.N', 'WYN.N', 'WYNN.O', 'X.N', 'XEC.N', 'XEL.N', 'XLNX.O', 'XMSR.O', 'XOM.N', 'XTO.N', 'YHOO.O', 'YUM.N', 'ZBRA.O', 'ZGEN.O', 'ZION.O', 'ZLC.N', 'ZMH.N', 'ZQK.N', 'ZRAN.O', 'ZUMZ.O', 'ZZ.N', 'BAK.N', 'CHZ.N', 'FULT.O', 'PTP.N', 'SKT.N', 'TSFG.O', 'WITS.O', 'BSTE.O', 'DSX.N', 'MOLXA.O', 'SYT.N', 'VMSI.O', 'ANSS.O', 'CRUS.O', 'CVBF.O', 'GRT.N', 'ILSE.O', 'IVC.N', 'MPEL.O', 'ABG.N', 'MOH.N', 'SVNT.O', 'BLK.N', 'GIL.N', 'MAA.N', 'PAC.N', 'RG.N', 'CBEY.O', 'LZB.N', 'OC.N', 'PJC.N', 'VOLC.O', 'CNH.N', 'MCRS.O', 'GHL.N', 'GLBC.O', 'QSII.O', 'SYX.N', 'TSS.N', 'USIH.O', 'NAVZ.OQ', 'TWGP.O', 'FCF.N', 'FRPT.O', 'BAP.N', 'ITC.N', 'KNXA.O', 'PFS.N', 'ESL.N', 'MPW.N', 'NAT.N', 'FRC.N', 'HLS.N', 'KSWS.O', 'SSW.N', 'TDY.N', 'PXR.N', 'TRST.O', 'VNDA.O', 'CCO.N', 'CEC.N', 'GISX.O', 'KRON.O', 'PTNR.O', 'IEX.N', 'NAK.A', 'ROIAK.O', 'SLXP.O', 'ARBA.O', 'HAE.N', 'HLIT.O', 'NEWP.O', 'ZNT.N', 'GFF.N', 'POR.N', 'QMAR.O', 'DECK.O', 'NATI.O', 'ZOLT.O', 'CCBL.O', 'MTX.N', 'BPT.N', 'NUS.N', 'AEO.N', 'ASPV.O', 'BBBB.O', 'BRKR.O', 'CW.N', 'GCA.N', 'MANH.O', 'RAH.N', 'UVV.N', 'ATR.N', 'AYR.N', 'ESS.N', 'IDC.N', 'SIM.A', 'VMED.O', 'CHTT.O', 'RAIL.O', 'RARE.O', 'SXT.N', 'WLSC.O', 'AMIS.O', 'HDB.N', 'HIBB.O', 'MINI.O', 'RWT.N', 'STKL.O', 'TCK.N', 'ASX.N', 'BKH.N', 'DJO.N', 'KNL.N', 'GGAL.O', 'TALX.O', 'TSL.N', 'HRH.N', 'XXIA.O', 'EIG.N', 'ONXX.O', 'SIGI.O', 'USTR.O', 'ARD.N', 'BOBE.O', 'IFSIA.O', 'ALSK.O', 'GFIG.O', 'SIMO.O', 'TEG.N', 'CENTA.O', 'ORA.N', 'PAA.N', 'AGYS.O', 'CBD.N', 'FS.N', 'HBG.N', 'HE.N', 'NPO.N', 'RJET.O', 'GLDN.O', 'SBGI.O', 'TRV.N', 'ARAY.O', 'NCMI.O', 'NETC.O', 'AB.N', 'CHKP.O', 'CPRT.O', 'DB.N', 'FIG.N', 'FLA.N', 'HNI.N', 'LOGI.O', 'MTU.N', 'RRGB.O', 'TI.N', 'CBM.N', 'CEDC.O', 'FSS.N', 'SMRT.O', 'SVR.N', 'TDS.A', 'VTIV.O', 'XTXI.O', 'ALJ.N', 'OPXT.O', 'PGNX.O', 'AZPN.O', 'NTO.A', 'SEH.N', 'AGO.N', 'JRN.N', 'PTMK.O', 'SIGM.O', 'XNPT.O', 'BARE.O', 'BKMU.O', 'MYE.N', 'PRSP.O', 'PRXL.O', 'VARI.O', 'SJI.N', 'VGR.N', 'ANAD.O', 'BPFH.O', 'UDR.N', 'UMPQ.O', 'USNA.O', 'ACO.N', 'RECN.O', 'DRTE.O', 'CSL.N', 'CVLT.O', 'ISLN.O', 'SGG.O', 'WNG.N', 'WRES.O', 'MEG.N', 'PIR.N', 'USBE.O', 'HNP.N', 'JAS.N', 'BBND.O', 'CYCL.O', 'FCSX.O', 'RNO.A', 'SMSC.O', 'DNDN.O', 'GHCI.O', 'JOSB.O', 'NR.N', 'ROSE.O', 'BR.N', 'NEU.N', 'RIN.O', 'RT.N', 'NICE.O', 'CPHD.O', 'CVNS.O', 'PBCT.O', 'TNH.N', 'CGNX.O', 'RESP.O', 'BAS.N', 'JAKK.O', 'EVVV.O', 'KTO.N', 'MNG.A', 'NM.N', 'FMBI.O', 'WPP.N', 'CELL.O', 'LOOP.O', 'NUVA.O', 'PBH.N', 'FELE.O', 'ACGL.O', 'CNQR.O', 'INTL.O', 'SUSQ.O', 'TFX.N', 'MAFB.O', 'PFGC.O', 'SPSS.O', 'ADVS.O', 'ALY.N', 'CMTL.O', 'OMCL.O', 'SNCR.O', 'NILE.O', 'AFSI.O', 'GEF.N', 'PCS.N', 'WIRE.O', 'CKXE.O', 'DAR.A', 'GB.N', 'SRCL.O', 'TFSL.O', 'VAS.N', 'DAL.N', 'EEP.N', 'RCI.N', 'BWLD.O', 'EDO.N', 'GLNG.O', 'HLTH.O', 'LNY.N', 'SPAR.O', 'IBKR.O', 'RCNI.O', 'ACM.N', 'ARUN.O', 'CUP.A', 'ENB.N', 'M.N', 'SFG.N', 'CLR.N', 'SRR.N', 'DCP.N', 'IART.O', 'ROC.N', 'NWA.N', 'CASY.O', 'ALE.N', 'CTR.N', 'HAIN.O', 'RRR.N', 'TASR.O', 'ENS.N', 'IHS.N', 'BCSI.O', 'CNK.N', 'DCM.N', 'FDP.N', 'VMI.N', 'FTEK.O', 'SWIR.O', 'THRX.O', 'APOG.O', 'CLRK.O', 'LLNW.O', 'BRCD.O', 'MIC.N', 'FRZ.N', 'PAG.N', 'DNN.A', 'LYG.N', 'SA.A', 'DFS.N', 'ESRX.O', 'SWHC.O', 'ASTSF.O', 'UPL.N', 'HAYN.O', 'IDCC.O', 'SLT.N', 'HEES.O', 'BX.N', 'CME.N', 'MOD.N', 'NETL.O', 'PHLY.O', 'TRMK.O', 'MHGC.O', 'SLF.N', 'XRTX.O', 'ARRO.O', 'AXCA.O', 'MGG.N', 'VLCM.O', 'WTNY.O', 'CATY.O', 'HCSG.O', 'PFWD.O', 'TPP.N', 'WABC.O', 'EXH.N', 'MDTH.O', 'ACIW.O', 'BXS.N', 'ETE.N', 'SBNY.O', 'GLYT.O', 'TR.N', 'GBCI.O', 'EXM.N', 'CLNE.O', 'SJR.N', 'TD.N', 'NSH.N', 'PLMD.O', 'CPNO.O', 'WBC.N', 'GIB.N', 'SMDI.O', 'BNS.N', 'OWW.N', 'AUXL.O', 'JDAS.O', 'NHWK.O', 'RDEN.O', 'IMO.A', 'BBL.N', 'BMO.N', 'MMP.N', 'PWRD.O', 'LULU.O', 'FCTR.O', 'PRFT.O', 'PSS.N', 'PSO.N', 'G.N', 'CXO.N', 'TKS.N', 'FIX.N', 'EJ.N', 'MASI.O', 'WX.N', 'MELI.O', 'MPWR.O', 'PBT.N', 'VIVO.O', 'LIHR.O', 'MFA.N', 'HSII.O', 'MXIM.OQ', 'TWI.N', 'VMW.N', 'ZNH.N', 'CZZ.N', 'HALO.O', 'BDE.N', 'CHA.N', 'NPBC.O', 'HTCH.O', 'PAET.O', 'MWE.N', 'HCP.N', 'ANW.N', 'VR.N', 'CN.N', 'SCUR.O', 'SGEN.O', 'TRAK.O', 'STAR.O', 'CUK.N', 'DNP.N', 'EXAR.O', 'KNDL.O', 'SCR.N', 'ZINC.O', 'DAR.N', 'TGB.A', 'INFN.O', 'CLMS.O', 'TIVO.O', 'LINE.O', 'OSTK.O', 'KEG.N', 'SPRD.O', 'TDC.N', 'SFLY.O', 'AMAG.O', 'FALC.O', 'UIC.N', 'UMC.N', 'ATHN.O', 'GSOL.O', 'HBHC.O', 'JRJC.O', 'GTLS.O', 'INCY.O', 'BKI.N', 'CETV.O', 'MDG.N', 'MTRX.O', 'PBKS.O', 'AZK.A', 'WTFC.O', 'CTCT.O', 'HTX.N', 'POWI.O', 'CM.N', 'CSR.N', 'NZ.P', 'PDGI.O', 'SNWL.O', 'CYBS.O', 'EPE.N', 'IOC.A', 'TLEO.O', 'EHTH.O', 'GLG.N', 'PCX.N', 'DFT.N', 'MBFI.O', 'MFN.A', 'MFG.N', 'RNOW.O', 'CVI.N', 'UBSI.O', 'LFT.N', 'NS.N', 'ULTA.O', 'TNP.N', 'HTH.N', 'MV.N', 'CBZ.N', 'DDUP.O', 'CISG.O', 'GA.N', 'TOC.N', 'TESO.O', 'AMCN.O', 'GRO.N', 'TT.N', 'APPX.O', 'MMR.N', 'NPD.N', 'FUN.N', 'TDG.N', 'IDEV.O', 'IVZ.N', 'OZM.N', 'CPTS.O', 'CBU.N', 'EPB.N', 'HGT.N', 'PAR.P', 'CCC.N', 'JAVA.O', 'AKRX.O', 'MWA/B.N', 'PBR/A.N', 'RDS/B.N', 'TRAD.O', 'ARTG.O', 'TRE.A', 'COHR.O', 'EROC.O', 'HPY.N', 'PUK.N', 'TBI.N', 'CSIQ.O', 'PANL.O', 'PRGS.O', 'TY.N', 'XIDE.O', 'CKP.N', 'RSTI.O', 'ABAX.O', 'LNN.N', 'SDTH.O', 'APL.N', 'BTI.A', 'CALM.O', 'RIGL.O', 'ARA.N', 'LMNX.O', 'PCZ.N', 'SYNT.O', 'XIN.N', 'CBSH.O', 'BKE.N', 'KBW.N', 'ULTI.O', 'VSAT.O', 'AMCC.O', 'ACC.N', 'CMO.N', 'IDXX.O', 'ACOR.O', 'QDEL.O', 'SJT.N', 'MATR.O', 'NDSN.O', 'NVO.N', 'WGOV.O', 'SATS.O', 'TGI.N', 'DISH.O', 'ROL.N', 'UMBF.O', 'ANH.N', 'SCSC.O', 'SONO.O', 'BIP.N', 'NWN.N', 'UFPI.O', 'ATN.N', 'HTE.N', 'MCY.N', 'STC.N', 'WRLD.O', 'CSGP.O', 'ASTE.O', 'HURN.O', 'RBA.N', 'VECO.O', 'FDG.N', 'MANT.O', 'SDA.N', 'AWI.N', 'BLOG.O', 'NWEC.O', 'DLTR.O', 'THS.N', 'MRB.A', 'ELS.N', 'BLT.N', 'DPM.N', 'LECO.O', 'SUP.N', 'ASIA.O', 'CLHB.O', 'IBOC.O', 'WRB.N', 'BPL.N', 'TRI.N', 'DCOM.O', 'NTAP.O', 'TC.N', 'TTMI.O', 'ANST.O', 'CA.O', 'RTIX.O', 'HRC.N', 'IIVI.O', 'SQM.N', 'ZOLL.O', 'PM.N', 'SNX.N', 'V.N', 'DAN.N', 'VQ.N', 'HMSY.O', 'TGP.N', 'GPOR.O', 'JRCC.O', 'GPK.N', 'HELE.O', 'KOP.N', 'SCOR.O', 'HTV.N', 'MXB.N', 'FEED.O', 'PLM.A', 'KALU.O', 'SLH.N', 'IPI.N', 'AWK.N', 'IBI.N', 'PACW.O', 'STEC.O', 'ZEUS.O', 'MLNX.O', 'GTE.A', 'PPO.N', 'BEXP.O', 'L.N', 'NYX.N', 'SPSX.O', 'TRLG.O', 'CFX.N', 'ARLP.O', 'LAYN.O', 'SFSF.O', 'IPHS.O', 'PVTB.O', 'SWIM.O', 'CME.O', 'ESMK.O', 'EZPW.O', 'PMC.N', 'SDXC.O', 'CYBX.O', 'FRED.O', 'ALTH.O', 'APWR.O', 'ARMH.O', 'CRESY.O', 'HTS.N', 'LMDIA.O', 'LO.N', 'SQNM.O', 'TWTC.O', 'WRC.N', 'FSYS.O', 'FOR.N', 'CPL.N', 'TCL.N', 'TII.N', 'ABI.N', 'KOF.N', 'SAN.N', 'SNI.N', 'ACAS.O', 'BABY.O', 'ROCK.O', 'ATVI.O', 'SNN.N', 'BOKF.O', 'LPS.N', 'NGD.A', 'EURX.O', 'GTIV.O', 'MNTA.O', 'ACE.N', 'VVUS.O', 'FINL.O', 'FSP.A', 'ERII.O', 'PRA.N', 'CNMD.O', 'MDVN.O', 'SBP.N', 'BLX.N', 'ALGT.O', 'LHCG.O', 'ABMD.O', 'SWSI.O', 'CRH.N', 'NTCT.O', 'HS.N', 'IRS.N', 'AAWW.O', 'ISCA.O', 'RAX.N', 'IIJI.O', 'HSNI.O', 'IILG.O', 'TKTM.O', 'BBEP.O', 'OKS.N', 'SF.N', 'GEOY.O', 'TIa.N', 'TCBI.O', 'MDRXD.O', 'RGAa.N', 'LGCY.O', 'OTTR.O', 'IRET.O', 'KELYA.O', 'NWE.N', 'TDS.N', 'ADP.O', 'EPIQ.O', 'KB.N', 'MATW.O', 'QCOR.O', 'SPWRA.O', 'CHSI.O', 'SB.N', 'TYL.N', 'VOCS.O', 'RGAb.N', 'GEL.A', 'KMR.N', 'SLGN.O', 'DISCK.O', 'SPH.N', 'BPZ.A', 'EGP.N', 'SPWRB.O', 'FMS.N', 'LUFK.O', 'DISCA.O', 'EBS.N', 'CCRN.O', 'DAC.N', 'PETS.O', 'AMSG.O', 'OGZP.O', 'SHLM.O', 'UGP.N', 'HITT.O', 'ISYS.O', 'AVAV.O', 'RGA.N', 'EBRb.N', 'MORN.O', 'AFAM.O', 'LAB.N', 'SSTI.O', 'LG.N', 'NPSP.O', 'WMZ.N', 'DMND.O', 'CFL.N', 'CPLA.O', 'SSS.N', 'RBS.N', 'SHG.N', 'MWW.N', 'CLH.N', 'EGY.N', 'GERN.O', 'JACK.O', 'ODSY.O', 'HOTT.O', 'APEI.O', 'EBR.N', 'NVE.N', 'SONE.O', 'AXYS.O', 'JWa.N', 'DRRX.O', 'FGP.N', 'MYL.O', 'AIPC.O', 'TSYS.O', 'EDE.N', 'EZCH.O', 'HOGS.O', 'LOPE.O', 'CBRL.O', 'AMSF.O', 'MDAS.O', 'OPTR.O', 'MMSI.O', 'RIG.N', 'ENZN.O', 'MAXY.O', 'SGTZY.O', 'CLWR.O', 'MOGa.N', 'ENL.N', 'MD.N', 'SMA.N', 'CRXL.O', 'IX.N', 'CML.P', 'CYPB.O', 'LTC.N', 'STBA.O', 'WES.N', 'ASEI.O', 'DWA.O', 'CIR.N', 'MWA.N', 'SXL.N', 'TISI.O', 'MHLD.O', 'MIM.N', 'PNFP.O', 'APU.N', 'GBG.A', 'LANC.O', 'CPKI.O', 'DROOY.O', 'EQT.N', 'FWLT.O', 'BJRI.O', 'LINC.O', 'AHGP.O', 'IMGN.O', 'CRA.O', 'GXDX.O', 'MIDD.O', 'MIG.N', 'PEGA.O', 'CPSI.O', 'IOC.N', 'MJN.N', 'WG.N', 'ARST.O', 'OSIR.O', 'ERIE.O', 'MYRG.O', 'TRH.N', 'TWC.N', 'VLTR.O', 'ETQ.A', 'MNRO.O', 'IBKC.O', 'AGII.O', 'NBTB.O', 'CTRN.O', 'TWX.N', 'ITU.N', 'TRK.N', 'AGCO.N', 'CACI.N', 'HI.N', 'APP.A', 'DIN.N', 'GHDX.O', 'TNS.N', 'ASR.N', 'PSEC.O', 'ITUB.N', 'CFSG.O', 'FIRE.O', 'BPI.N', 'OMGI.O', 'RST.N', 'IPXL.O', 'VIT.N', 'WAL.N', 'GGWPQ.OQ', 'SEP.N', 'UIL.N', 'VALE.N', 'TPC.N', 'FFBC.O', 'DGI.N', 'CDE.N', 'SWI.N', 'VITA.O', 'UFS.N', 'DEP.N', 'FUQI.O', 'EVEP.O', 'GOV.N', 'EGOV.O', 'HRBN.O', 'PLX.A', 'ENP.N', 'TEL.N', 'AIG.N', 'IR.N', 'MSCS.O', 'PDA.N', 'SVM.A', 'NOVN.O', 'AAV.N', 'DGW.N', 'HA.O', 'MDSO.O', 'ROVI.O', 'FICO.N', 'LOGM.O', 'CYD.N', 'ENOC.O', 'KSP.N', 'EMS.N', 'MITI.O', 'SU.N', 'ROY.A', 'BCRX.O', 'GLDD.O', 'KLIC.O', 'ISPH.O', 'WCRX.O', 'ITWO.O', 'NVAX.O', 'IMMU.O', 'MAT.O', 'TGH.N', 'ACN.N', 'EM.N', 'STWD.N', 'ANV.A', 'BZ.N', 'CBE.N', 'BGS.N', 'VPRT.O', 'RINO.O', 'ABVT.N', 'OPEN.O', 'BMC.O', 'FNSR.O', 'THI.N', 'ATLS.O', 'IRDM.O', 'JNPR.N', 'TLVT.O', 'CFN.N', 'SHAW.N', 'AONE.O', 'ART.N', 'CY.O', 'GAME.O', 'SEM.N', 'CML.N', 'GORO.OB', 'TLCR.O', 'EDMC.O', 'EBIX.O', 'PAR.N', 'UIS.N', 'BSBR.N', 'VRSK.O', 'NZ.N', 'NSU.A', 'RA.N', 'MRK.N', 'CRIC.O', 'PEET.O', 'REV.N', 'MED.N', 'AGAM.O', 'CADX.O', 'PSE.N', 'BRFS.N', 'PODD.O', 'DOLE.N', 'TCLP.O', 'BBVA.N', 'RHB.N', 'ARTC.O', 'CAAS.O', 'VSI.N', 'FBR.N', 'WPRT.O', 'ACOM.O', 'WBD.N', 'WBMD.O', 'CEL.N', 'MU.O', 'KT.N', 'ROIC.O', 'RUE.O', 'TW.N', 'FTNT.O', 'CIT.N', 'CLD.N', 'CVE.N', 'SVN.N', 'CLMT.O', 'FGXI.O', 'NWBI.O', 'AIRV.O', 'BACPRS.N', 'OSIS.O', 'CCM.N', 'BTE.N', 'PGH.N', 'PGN.N', 'PVG.N', 'PVX.N', 'PWE.N', 'SUNH.O', 'VIP.N', 'MF.N', 'VNR.N', 'KRA.N', 'XRA.A', 'DXCM.O', 'SCCO.N', 'COLB.O', 'PPC.N', 'BRKb.N', 'NMM.N', 'SCHW.N', 'SWM.N', 'AOL.N', 'SYA.N', 'NGLS.N', 'SMCI.O', 'LL.N', 'DEXO.N', 'CVVT.O', 'SPMD.O', 'EXXI.O', 'PDM.N', 'GNRC.N', 'GRM.N', 'QNST.O', 'IUSA.O', 'GPRE.O', 'SWK.N', 'BNE.N', 'TMH.N', 'NOG.A', 'ST.N', 'CQP.A', 'CODI.O', 'ABX.N', 'AEM.N', 'BAM.N', 'COT.N', 'EGO.A', 'MEOH.O', 'PDS.N', 'RY.N', 'SDRL.N', 'PRI.N', 'CAK.A', 'OCLR.O', 'IVR.N', 'SXE.N', 'CNO.N', 'FRG.A', 'KS.N', 'BKCC.O', 'FPO.N', 'VRUS.O', 'ALTE.O', 'CLGX.N', 'ETFC.O', 'EXPR.N', 'SM.N', 'NEE.N', 'ACTG.O', 'AEC.N', 'DDAIF.OB', 'FSR.N', 'MSCI.N', 'FAF.N', 'PWER.O', 'DTEGY.OB', 'OPK.A', 'WNC.N', 'ENTR.O', 'GRMN.O', 'QEP.N', 'STX.O', 'OAS.N', 'MFB.N', 'EC.N', 'IGTE.O', 'SSCC.N', 'CFFN.O', 'NXTM.O', 'ALR.N', 'RPT.N', 'TSLA.O', 'ORIT.O', 'PEB.N', 'SMRC.O', 'ATAC.O', 'RADS.O', 'N.N', 'AEZ.A', 'EVR.N', 'IPGP.O', 'KOG.A', 'MCP.N', 'TRS.O', 'NXPI.O', 'THM.A', 'JKS.N', 'MVG.A', 'MMYT.O', 'ARIA.O', 'RDWR.O', 'FNGN.O', 'GFIG.N', 'QLIK.O', 'SIG.N', 'INET.O', 'RIO.N', 'GEL.N', 'HLTOY.O', 'VHC.A', 'BPO.N', 'ERF.N', 'RLD.N', 'NKA.N', 'UAL.O', 'CCE.N', 'CODI.N', 'LPSN.O', 'MIPS.O', 'LYB.N', 'PAM.N', 'JNY.N', 'SBRA.O', 'SIGA.O', 'MY.N', 'SNIC.O', 'ELT.N', 'CCME.O', 'TNK.N', 'EGO.N', 'UAL.N', 'ATSG.O', 'CODE.N', 'EPAY.O', 'VSTO.OB', 'PACB.O', 'BPZ.N', 'PCH.O', 'SGTPY.UNKNOWN', 'HAS.O', 'CMRE.N', 'IMAX.O', 'ONE.N', 'BGCP.O', 'BRLI.O', 'SODA.O', 'CAR.O', 'MAKO.O', 'BSFT.O', 'TGA.O', 'CFFND.O', 'IL.N', 'EXK.A', 'SMFG.N', 'GM.N', 'CIS.N', 'VC.N', 'AIXG.O', 'AMRN.O', 'ERJ.N', 'PRISb.N', 'VRA.O', 'AAN.N', 'GBX.N', 'DANG.N', 'YOKU.N', 'ORLY.O', 'WWD.O', 'ASNA.O', 'MSI.N', 'CHMT.N', 'CHT.N', 'GLF.N', 'SWFT.N', 'HYC.N', 'KEM.N', 'IMAX.N', 'WSDT.O', 'GEOI.O', 'GTY.N', 'UTEK.O', 'CLDA.O', 'LRN.N', 'AZC.A', 'FNMA.OQ', 'BKU.N', 'LNCE.O', 'PNNT.O', 'AREX.O', 'AWH.N', 'IRBT.O', 'LXU.N', 'FN.N', 'REPYY.OB', 'TDSC.O', 'MERC.O', 'ANN.N', 'SGI.O', 'SIFY.O', 'TAL.N', 'COV.N', 'YPF.N', 'AXU.A', 'HOLI.O', 'MHR.N', 'MTOR.N', 'HCA.N', 'NTSP.O', 'TRGP.N', 'CEVA.O', 'CXS.N', 'HII.N', 'APO.N', 'QIHU.N', 'TFM.O', 'CALX.N', 'DDD.N', 'CLNY.N', 'TITN.O', 'UAM.N', 'UAN.N', 'ARCO.N', 'SUBCY.O', 'C.N', 'VNET.O', 'AVEO.O', 'KRO.N', 'TNAV.O', 'RP.O', 'AIMC.O', 'GDOT.N', 'WFM.O', 'PSMT.O', 'MOS.N', 'SEMG.N', 'GORO.A', 'ACW.N', 'HF.N', 'KOS.N', 'PRIM.O', 'RLJ.N', 'AL.N', 'CBOE.O', 'PLD.N', 'ZIP.O', 'CATO.N', 'LNKD.N', 'CHTR.O', 'SLRC.O', 'AUQ.N', 'BAH.N', 'YNDX.O', 'CWT.N', 'FSL.N', 'SZYM.O', 'CAVM.O', 'SFSF.N', 'EXAM.N', 'IRWD.O', 'SIX.N', 'SREV.O', 'PCYC.O', 'FIO.N', 'LAVA.O', 'HFC.N', 'P.N', 'KOG.N', 'AMN.N', 'ARJ.N', 'AMCX.O', 'ELGX.O', 'MPC.N', 'PVH.N', 'SPB.N', 'BIN.N', 'IDIX.O', 'ENSG.O', 'SPPI.O', 'GNC.N', 'ESC.N', 'CNSL.O', 'GTAT.O', 'LQDT.O', 'RGP.N', 'XG.A', 'MKTX.O', 'RGR.N', 'RL.N', 'DNKN.O', 'CJES.N', 'CMVT.OB', 'OPK.N', 'APAC.O', 'DTSI.O', 'TRGT.O', 'HEI.N', 'KMPR.N', 'CMVT.O', 'AIBYY.O', 'DLLR.O', 'OXM.N', 'KAMN.O', 'WETF.O', 'CYS.N', 'GIII.O', 'CALP.O', 'CLW.N', 'MXWL.O', 'CUBE.N', 'TYPE.O', 'ALKS.O', 'KFRC.O', 'LPR.N', 'WAIR.N', 'BEAM.N', 'LSTZA.O', 'CBG.N', 'JBT.N', 'VIV.N', 'IRE.N', 'SPWR.O', 'FBHS.N', 'OPNT.O', 'AEGN.O', 'LAD.N', 'LVLT.N', 'MAIN.N', 'ITT.N', 'VIAB.O', 'PMT.N', 'SLM.O', 'XYL.N', 'INHX.O', 'MGI.N', 'RPXC.O', 'AAT.N', 'RATE.N', 'XLS.N', 'EA.O', 'GRPN.O', 'HITK.O', 'WEN.O', 'ACTV.N', 'PB.N', 'VAC.N', 'EXAS.O', 'FTK.N', 'NEOG.O', 'TXN.O', 'FRAN.O', 'DLPH.N', 'JCOM.O', 'ACHN.O', 'EXPE.O', 'OSUR.O', 'NBIX.O', 'KORS.N', 'TRIP.O', 'ZNGA.O', 'IPCM.O', 'SUI.N', 'SXC.N', 'CHKR.N', 'ALLT.O', 'AWAY.O', 'BBCN.O', 'INVN.N', 'JOY.N', 'MKTG.O', 'MNST.O', 'JAZZ.O', 'SAVE.O', 'UBNT.O', 'WPX.N', 'KNXA.N', 'WAFD.O', 'BRGYY.OB', 'MUX.N', 'TNGO.O', 'JIVE.O', 'HTGC.O', 'FLT.N', 'CSOD.O', 'Z.O', 'OCZ.O', 'POST.N', 'RSE.N', 'BCPC.O', 'DX.N', 'KNOL.O', 'ADI.O', 'AVD.N', 'SLCA.N', 'DUF.N', 'CRBC.O', 'INXN.N', 'CALL.O', 'YELP.N', 'AMTD.N', 'LPI.N', 'CZR.O', 'AON.N', 'GWRE.N', 'HTGC.N', 'ACCO.N', 'CONN.O', 'ALSN.N', 'QRE.N', 'ACAT.O', 'RUSHA.O', 'LKQ.O', 'SA.N', 'MOV.N', 'TEVA.N', 'FET.N', 'ANGI.O', 'ARCT.O', 'HTSI.N', 'MFRM.O', 'MM.N', 'RNDY.N', 'VNTV.N', 'WDC.O', 'TREX.N', 'SPLK.O', 'TUMI.N', 'PSX.N', 'UNS.N', 'AFFY.O', 'HPP.N', 'LF.N', 'BCEI.N', 'EXLS.O', 'ACHC.O', 'RFP.N', 'AMTG.N', 'BPOP.O', 'IMPV.N', 'INGR.N', 'RPAI.N', 'ET.N', 'FB.O', 'BCOR.O', 'MODL.O', 'SHPG.O', 'RXN.N', 'PDCE.O', 'LPLA.O', 'SSNC.O', 'ZIOP.O', 'ABCO.O', 'HSH.N', 'PCRX.O', 'DUK.N', 'KW.N', 'PKT.O', 'MAPP.O', 'CTRX.O', 'BKW.N', 'SN.N', 'BV.O', 'SUSS.O', 'ELLI.N', 'GRFS.O', 'AVG.N', 'PES.N', 'TROX.N', 'BTH.N', 'BSAC.N', 'INFI.O', 'FIVE.O', 'TRQ.N', 'PANW.N', 'DENN.O', 'NOW.N', 'STSA.O', 'NTI.N', 'FBP.N', 'FNV.N', 'SAND.A', 'AWR.N', 'PVR.N', 'BLMN.O', 'DTV.O', 'MITT.N', 'STAG.N', 'BIOS.O', 'ARRY.O', 'TOO.N', 'CHSP.N', 'EPL.N', 'MATX.N', 'VHS.N', 'SNTS.O', 'EVER.N', 'BLDR.O', 'COR.N', 'RHP.N', 'MDLZ.O', 'WPC.N', 'BLOX.N', 'ASPS.O', 'SPNC.O', 'SRC.N', 'FBC.N', 'LFL.N', 'THC.N', 'ADT.N', 'NYCB.N', 'SRPT.O', 'WAC.A', 'BSMX.N', 'KRFT.O', 'HLSS.O', 'SHFL.O', 'DYN.N', 'PRLB.N', 'RLGY.N', 'SHF.N', 'VRNT.O', 'WXS.N', 'LNCO.O', 'WDAY.N', 'AEGR.O', 'DWRE.N', 'BNNY.N', 'MRC.N', 'EPR.N', 'EXL.N', 'INFY.N', 'CRAY.O', 'SANM.O', 'GLRE.O', 'GT.O', 'HTWR.O', 'IRET.N', 'BANR.O', 'LSI.O', 'CSU.N', 'ASML.O', 'PPC.O', 'ETN.N', 'ECPG.O', 'RKUS.N', 'PKY.N', 'EGHT.O', 'SSYS.O', 'USNA.N', 'MHO.N', 'BERY.N', 'TILE.O', 'ACCL.O', 'AIRM.O', 'ABBV.N', 'PBF.N', 'SCTY.O', 'BDBD.O', 'SBY.N', 'SMP.N', 'WAC.N', 'VVI.N', 'SUSS.N', 'JE.N', 'INN.N', 'DORM.O', 'NFBK.O', 'AGRO.N', 'PMTC.O', 'AXLL.N', 'STNG.N', 'STRZA.O', 'ICLR.O', 'MX.N', 'CVRR.N', 'HAFC.O', 'GMED.N', 'NCLH.O', 'AAMRQ.OB', 'CLDX.O', 'TPH.N', 'TRLA.N', 'ZTS.N', 'KERX.O', 'LMCA.O', 'EEQ.N', 'PACT.O', 'MGAM.O', 'SBGL.N', 'CCG.N', 'ARI.N', 'WEX.N', 'WMC.N', 'FNMA.OB', 'VNR.O', 'GBDC.O', 'DDC.N', 'LOCK.N', 'LTD.N', 'ARII.O', 'CNCO.N', 'TMUS.N', 'AVIV.N', 'SSP.N', 'PF.N', 'RAS.N', 'ACAD.O', 'LSE.N', 'PCRFY.N', 'AMAP.O', 'OFG.N', 'FANG.O', 'TMHC.N', 'EVTC.N', 'MDXG.O', 'YY.O', 'FRGI.O', 'FMCC.OB', 'SEAS.N', 'CST.N', 'AIBYY.OB', 'AMBC.O', 'IMOS.O', 'NXST.O', 'OLED.O', 'WWAV.N', 'XONE.O', 'ARPI.N', 'NBG.N', 'CLVS.O', 'BCC.N', 'SUNE.N', 'TSRO.O', 'TTS.O', 'CBF.O', 'LGND.O', 'FOSL.O', 'HHC.N', 'PFPT.O', 'LBTYA.O', 'LBTYK.O', 'CSG.N', 'CRWN.O', 'PGEM.N', 'THR.N', 'BTG.A', 'MTDR.N', 'WAGE.N', 'MEI.N', 'EPAM.N', 'LITB.N', 'SIR.N', 'CHUY.O', 'AVNR.O', 'FOX.O', 'FOXA.O', 'ORAN.N', 'OUTR.O', 'COTY.N', 'CSCTF.OB', 'TU.N', 'NWS.O', 'NWSA.O', 'GOGO.O', 'MNK.N', 'S.N', 'SAIA.O', 'COLE.N', 'HDS.O', 'ONB.O', 'BBRY.O', 'NDLS.O', 'ENV.N', 'KCG.N', 'CSTE.O', 'CPLP.O', 'BOFI.O', 'XOOM.O', 'UCBI.O', 'NOAH.N', 'AMH.N', 'SFM.O', 'DF.N', 'XON.N', 'FI.N', 'RPTP.O', 'SALE.O', 'EVHC.N', 'TPRE.N', 'SGNT.O', 'BITA.N', 'MKTO.O', 'MUSA.N', 'PNG.N', 'FLTX.N', 'SSNI.N', 'SYRG.A', 'HOMB.O', 'MAR.O', 'ALEX.N', 'KYTH.O', 'LDOS.N', 'TPLM.A', 'MOVE.O', 'DYAX.O', 'FCS.O', 'THRM.O', 'CNHI.N', 'CAMP.O', 'FEYE.O', 'LIN.N', 'HSBC.N', 'GIMO.N', 'PBPB.O', 'SFXE.O', 'AR.N', 'CXP.N', 'FF.N', 'LB.N', 'PTC.O', 'TRNX.O', 'GLPI.O', 'RDA.O', 'CSTM.N', 'FLDM.O', 'FUEL.O', 'AAL.O', 'DATA.N', 'ABEV.N', 'CSII.O', 'STL.N', 'CDW.O', 'ICE.N', 'COMM.O', 'EIGI.O', 'SIRI.O', 'QIWI.O', 'TLLP.N', 'BRX.N', 'NE.N', 'STAR.N', 'DOX.O', 'TCS.N', 'CIEN.N', 'NHI.N', 'ONVO.A', 'TWTR.N', 'WIBC.O', 'ESRT.N', 'SSLT.N', 'ALLE.N', 'SGMO.O', 'SAIC.N', 'PTLA.O', 'AGI.N', 'ICPT.O', 'PEGI.O', 'PRGO.N', 'TXTR.N', 'AMBA.O', 'NCS.N', 'XPO.N', 'ANAC.O', 'ENT.O', 'GEVA.O', 'VRTU.O', 'AGIO.O', 'EPZM.O', 'GLOG.N', 'ZLTQ.O', 'INSM.O', 'VLRS.N', 'ECOL.O', 'NVDQ.O', 'RRTS.N', 'TXMD.A', 'NLNK.O', 'LPT.N', 'ATHL.N', 'CNVR.O', 'ORCL.N', 'SC.N', 'SWAY.N', 'OGS.N', 'RICE.N', 'LCI.N', 'GWPH.O', 'MITL.O', 'ECOM.N', 'WWE.N', 'VOD.O', 'PTCT.O', 'KATE.N', 'DEPO.O', 'ENDP.O', 'INSY.O', 'CACQ.O', 'MDCA.O', 'ENLK.N', 'KN.N', 'GMCR.O', 'RCPT.O', 'CRTO.O', 'RSPP.N', 'APAM.N', 'MGI.O', 'PLUG.O', 'NAVI.O', 'DECK.N', 'VOYA.N', 'NMBL.N', 'GSAT.A', 'KING.N', 'HAWKB.O', 'HCT.O', 'NMFC.N', 'ALLY.N', 'BURL.N', 'ARCB.O', 'NDZ.N', 'FSIC.N', 'SABR.O', 'WB.O', 'ISBC.O', 'OPHT.O', 'SNV.N', 'HMHC.O', 'LQ.N', 'ARMK.N', 'CPF.N', 'PAH.N', 'XLRN.O', 'IMS.N', 'PNR.N', 'JMEI.N', 'RCAP.N', 'TRUE.O', 'FFIN.O', 'RLYP.O', 'TBPH.O', 'TOUR.O', 'KFX.O', 'JD.O', 'LE.O', 'PE.N', 'BFR.N', 'KPTI.O', 'NGHC.O', 'DNOW.N', 'TIME.N', 'WFT.N', 'AXDX.O', 'WSTC.O', 'CMRX.O', 'I.N', 'BLUE.O', 'GEO.N', 'YRCW.O', 'BEL.N', 'FNF.N', 'LEJU.N', 'MRD.O', 'MRKT.O', 'CONE.O', 'ECR.N', 'FNFV.N', 'WUBA.N', 'MIK.O', 'RYAM.N', 'SSE.N', 'TMST.N', 'NSAM.N', 'GPRO.O', 'XRS.N', 'EQC.N', 'STAY.N', 'KNDI.O', 'ATHM.N', 'SERV.N', 'HCLP.N', 'KRG.N', 'LOCO.O', 'GRUB.N', 'XNET.O', 'CTLT.N', 'SYF.N', 'ODP.O', 'MOBL.O', 'VEEV.N', 'TNET.N', 'BDSI.O', 'LTRPA.O', 'BXMT.N', 'HZNP.O', 'NVGS.N', 'PBA.N', 'ABY.O', 'ICL.N', 'SDLP.N', 'BABA.N', 'LTS.A', 'ANET.N', 'FCAU.N', 'CDK.O', 'CFG.N', 'CYBR.O', 'KITE.O', 'LMCK.O', 'QLYS.O', 'QVCA.O', 'TERP.O', 'VALEp.N', 'CMPR.O', 'ENTA.O', 'NRZ.N', 'VSLR.N', 'COUP.N', 'RGEN.O', 'SNR.N', 'PRAA.O', 'TRCO.N', 'ZEN.N', 'KEYS.N', 'RGLS.O', 'HYH.N', 'QSR.N', 'TYC.N', 'DCT.N', 'ESNT.N', 'LAMR.O', 'NYMT.O', 'MMLP.O', 'OUT.N', 'CLDT.N', 'TCPC.O', 'VWR.O', 'ASB.N', 'LBRDK.O', 'UBS.N', 'KANG.O', 'EXLP.O', 'CALD.O', 'EDR.N', 'AXTA.N', 'WBA.O', 'AMD.O', 'ANTM.N', 'QRVO.O', 'TECH.O', 'TTPH.O', 'VA.O', 'MGNX.O', 'ZU.O', 'PGRE.N', 'OMER.O', 'MORE.N', 'CCI.N', 'HTA.N', 'QUNR.O', 'FOLD.O', 'RDUS.O', 'TSEM.O', 'OVAS.O', 'BRP.N', 'DOC.N', 'KLXI.O', 'LC.N', 'MOMO.O', 'ESPR.O', 'ALDR.O', 'AAVL.O', 'GMLP.O', 'JUNO.O', 'CEMP.O', 'BNCL.O', 'FMI.O', 'NEWM.N', 'MTSI.O', 'IRM.N', 'PAYC.N', 'RIOM.N', 'UVE.N', 'MDT.N', 'UE.N', 'SONS.O', 'WPZ.N', 'PLAY.O', 'W.N', 'OA.N', 'PLKI.O', 'TW.O', 'DRII.N', 'ES.N', 'TAHO.N', 'ONDK.N', 'VSTO.N', 'CKEC.O', 'XHR.N', 'LNG.A', 'HDP.O', 'ON.O', 'ZAYO.N', 'TLMR.O', 'PRTA.O', 'RNG.N', 'ASPX.O', 'DPLO.N', 'LTXB.O', 'STOR.N', 'TVPT.N', 'AVAL.N', 'GPT.N', 'ZSPH.O', 'TKMR.O', 'GTN.N', 'SCMP.O', 'CIM.N', 'IGT.N', 'RTRX.O', 'CUDA.N', 'GDDY.N', 'CSAL.O', 'EURN.N', 'LEAF.N', 'HQY.O', 'ETSY.O', 'INCR.O', 'SEDG.O', 'SHAK.N', 'CPG.N', 'PBYI.N', 'FGEN.O', 'ECHO.O', 'KRNY.O', 'SAGE.O', 'CMCM.N', 'PCTY.O', 'TGTX.O', 'BOJA.O', 'HAWK.O', 'QTS.N', 'ZBH.N', 'NYLD.N', 'NVRO.N', 'CPGX.N', 'WRK.N', 'KHC.O', 'APLE.N', 'EGRX.O', 'TLN.N', 'LXRX.O', 'MYCC.N', 'IPHI.N', 'HIFR.N', 'HRTX.O', 'HT.N', 'ATRA.O', 'PSG.N', 'REXR.N', 'TGNA.N', 'WEC.N', 'BWXT.N', 'EPC.N', 'ITCI.O', 'SUPN.O', 'AJRD.N', 'FIT.N', 'SGYP.O', 'TREE.O', 'WIX.O', 'BXLT.N', 'WD.N', 'XENT.O', 'BLD.N', 'ENR.N', 'GCI.N', 'NWBO.O', 'SSTK.N', 'EROS.N', 'FMSA.N', 'ZG.O', 'REMY.O', 'ZOES.N', 'CYNO.O', 'DVAX.O', 'LDRH.O', 'FDML.O', 'INGN.O', 'VER.N', 'ARR.N', 'ELNK.O', 'VIAV.O', 'PYPL.O', 'SUM.N', 'STMP.O', 'TWOU.O', 'AVOL.N', 'RENT.O', 'BUFF.O', 'NK.O', 'AMRI.O', 'GLBL.O', 'LITE.O', 'RUN.O', 'NLSN.N', 'TDOC.N', 'CCP.N', 'ESND.O', 'CBPX.N', 'EBSB.O', 'OLLI.O', 'ONCE.O', 'TRU.N', 'UNVR.N', 'WING.O', 'VEDL.N', 'ACET.O', 'DHT.N', 'LIVN.O', 'AXON.N', 'FLOW.N', 'PRAH.O', 'TMUS.O', 'HCN.N', 'ADPT.N', 'CAA.N', 'PINC.O', 'GOOG.O', 'GOOGL.O', 'WMGI.O', 'AROC.N', 'VIRT.O', 'HPQ.N', 'MSGN.N', 'NRF.N', 'MXL.N', 'CAKE.O', 'ABCB.O', 'GWB.N', 'RYAAY.O', 'OMF.N', 'RVNC.O', 'PRTY.N', 'STE.N', 'FPRX.O', 'WEB.O', 'AAON.O', 'BW.N', 'PNK.O', 'HPE.N', 'CSFL.O', 'INOV.O', 'ADRO.O', 'CSX.O', 'IONS.O', 'CEQP.N', 'FCPT.N', 'SPTN.O', 'TIER.N', 'ENLC.N', 'INVA.O', 'CSRA.N', 'NRE.N', 'CHRS.O', 'NVCR.O', 'GSM.O', 'WBAI.N', 'SMLP.N', 'TEAM.O', 'TLRD.N', 'FCEa.N', 'RACE.N', 'AGR.N', 'ARRS.O', 'WLTW.O', 'RLI.N', 'LPG.N', 'SCAI.O', 'NLS.N', 'NEWR.N', 'RUTH.O', 'FRO.N', 'CHFC.O', 'HUBB.N', 'RUBI.N', 'HUBS.N', 'MNR.N', 'SHOP.N', 'ORBK.O', 'BANC.N', 'QUOT.N', 'SNI.O', 'IOSP.O', 'LADR.N', 'ORBC.O', 'USCR.O', 'XRAY.O', 'ALDW.N', 'MFS.N', 'BETR.N', 'TSE.N', 'CPE.N', 'IAC.O', 'LSXMA.O', 'LSXMK.O', 'CTRE.O', 'SFBS.O', 'SPXC.N', 'BNFT.O', 'SGRY.O', 'MRCY.O', 'YRD.N', 'INO.O', 'CVT.N', 'PSTG.N', 'SR.N', 'EBRYY.OB', 'FSM.N', 'UAc.N', 'SCLN.O', 'ENIA.N', 'FGL.N', 'OPB.O', 'SPGI.N', 'GKOS.N', 'DERM.O', 'ERI.O', 'MGP.N', 'ENIC.N', 'RRR.O', 'PFGC.N', 'SAAS.O', 'CPXX.O', 'NEO.O', 'AKG.A', 'GCP.N', 'OPK.O', 'AMPH.O', 'QUAD.N', 'SRG.N', 'YDKN.N', 'MCRB.O', 'NGVT.N', 'PLNT.N', 'ACIA.O', 'GATX.N', 'GOV.O', 'HPT.O', 'SIR.O', 'SNH.O', 'BMCH.O', 'SHEN.O', 'USFD.N', 'LILAK.O', 'BFAM.N', 'LILA.O', 'TOWN.O', 'CATM.O', 'HOPE.O', 'INFO.O', 'KTWO.O', 'XNCR.O', 'SQ.N', 'CERS.O', 'LGIH.O', 'FTV.N', 'HRI.N', 'NSA.N', 'FIZZ.O', 'XL.N', 'GBT.O', 'RAVN.O', 'ITGR.N', 'SFS.N', 'AM.N', 'CG.O', 'MPLX.N', 'NGL.N', 'NOMD.N', 'SHLX.N', 'VIPS.N', 'ANIP.O', 'LN.N', 'ININ.O', 'FOXF.O', 'FIVN.O', 'PSXP.N', 'AVXS.O', 'ALRM.O', 'GK.O', 'CFNL.O', 'RMP.N', 'TRTN.N', 'BPMC.O', 'FHB.O', 'IBP.N', 'RPD.O', 'RNST.O', 'CWEI.N', 'OSG.N', 'LBRDA.O', 'WPG.N', 'MTCH.O', 'DEA.N', 'HOME.N', 'EVH.N', 'DLTH.O', 'NEP.N', 'BOX.N', 'HASI.N', 'NPTN.N', 'APFH.N', 'TRNO.N', 'ILG.O', 'AERI.O', 'DVMT.N', 'CPPL.N', 'SFUN.N', 'BZUN.O', 'MIME.O', 'PTHN.N', 'AGNC.O', 'CORT.O', 'RRD.N', 'TEP.N', 'VVV.N', 'FCFS.N', 'PZE.N', 'FTS.N', 'VSM.N', 'ADC.N', 'MTL.N', 'TEDU.O', 'ARNC.N', 'XOG.O', 'CUBI.N', 'LFUS.O', 'YUMC.N', 'HMST.O', 'LVNTA.O', 'UA.N', 'FRTA.O', 'QCP.N', 'AIMT.O', 'CXW.N', 'LGFb.N', 'TECK.N', 'IMPV.O', 'CHGG.N', 'PAGP.N', 'TWLO.N', 'ZTO.N', 'ADNT.N', 'AA.N', 'SBCF.O', 'CDEV.O', 'BNCN.O', 'LEXEA.O', 'LW.N', 'MTGE.O', 'SITE.N', 'FCB.N', 'AMC.N', 'CVGW.O']
# sub = [i.split('.')[0] for i in assets]
# sub_asset = list(set(sub))

# test_data = marketDataGen(sub_asset, start='1/1/2017', end='12/31/2017')

# test_data.to_csv('/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data/test.csv')

In [44]:
test_data.shape

(529473, 9)

In [80]:
qcom = df.query("assetCode == 'QCOM'")['close']

In [36]:
X = csv.copy()

# substituting lambda
asset_close = X.groupby('assetCode')['close']
sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values        
            )

%time

CPU times: user 28 µs, sys: 1e+03 ns, total: 29 µs
Wall time: 6.91 µs


In [47]:
def moving_average(x, n=20):
    v = np.cumsum(x.values, dtype=float)
    v[n:] = v[n:] - v[:-n] 
    return v / n

In [48]:
X = csv.copy()

# substituting lambda
asset_close = X.groupby('assetCode')['close']
sm_20 = (
            asset_close.
            transform(moving_average)       
            )

%time

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [59]:
X['cum'] = X.groupby('assetCode')['close'].cumsum()

In [60]:
X.head()

,time,assetCode,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returnsOpenNextMktres10,cum
0,2014-01-16,QCOM,74.470001,74.720001,10263200.0,0.002814,0.013519,0.018914,0.011615,-0.015252,74.720001
1,2014-01-16,KEP,16.660000,16.719999,205300.0,0.009012,0.000000,0.017497,0.009650,-0.010187,16.719999
2,2014-01-16,NWS,17.059999,17.040001,506000.0,-0.006435,-0.011076,-0.048678,-0.044707,0.050676,17.040001
3,2014-01-16,BHP,66.570000,66.889999,5735900.0,0.029432,0.033296,-0.001494,-0.012391,0.013469,66.889999
4,2014-01-16,NGG,70.043671,69.934494,212800.0,0.001875,0.004530,-0.004361,-0.009308,-0.042414,69.934494


In [67]:
sma = X.groupby('assetCode')['cum'].transform(moving_average)

%time

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


In [64]:
def moving_average(x, n=20):
    v = x.values
    v[n:] = v[n:] - v[:-n] 
    return v / n

# This actually preserves the dimension

In [69]:
k = 2 / (20 +1)
k

0.09523809523809523

In [70]:
def ExpMovingAverage(values, window):
    """ Numpy implementation of EMA
    """
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a =  np.convolve(values, weights, mode='full')[:len(values)]
    a[:window] = a[window]
    return a

In [92]:
weights = np.exp(np.linspace(-1., 0., 5))

In [95]:
weights

array([0.11405072, 0.14644403, 0.18803785, 0.24144538, 0.31002201])

In [94]:
weights /= weights.sum()

In [96]:
test = qcom[:15].values
test

array([74.72000122, 74.73000336, 75.45999908, 75.77999878, 75.87000275,
       74.08000183, 73.19999695, 71.98999786, 71.12000275, 73.26000214,
       74.22000122, 73.31999969, 72.51000214, 72.37000275, 73.23999786])

In [97]:
a =  np.convolve(test, weights, mode='full')[:len(test)]

In [98]:
a

array([ 8.52187022, 19.46530893, 33.60021891, 51.7862988 , 75.14795262,
       75.1965093 , 75.15451051, 74.67198784, 73.82580865, 72.94753311,
       72.64185024, 72.49700878, 72.70031963, 73.2917332 , 73.29846413])

In [99]:
a[:5] = a[5]

In [100]:
a

array([75.1965093 , 75.1965093 , 75.1965093 , 75.1965093 , 75.1965093 ,
       75.1965093 , 75.15451051, 74.67198784, 73.82580865, 72.94753311,
       72.64185024, 72.49700878, 72.70031963, 73.2917332 , 73.29846413])

In [ ]:
# data is used as the base set
data = base.copy()

data >> \
        exTractTime(col='time', 
                    atts=['weekday', 'month', 'year', 'day', 'quarter'],
                    mthds=['month_name']) >> \
        toCatFeat(feats=['year', 'quarter']) >> \
        macdFeats() >> \
        bbSqueeze() >> \
        rsiFeats()
        
data.tail()


In [ ]:
# CHECK MY NULLS
(
    data[[col for col in data.columns if data[col].isnull().sum() > 0]].
    isnull().
    sum()
)




In [ ]:
X = data.copy()

X.dropna(inplace=True)
X.drop(['sq_min'], axis=1, inplace=True)
print('Dropped nulls from training set ONLY. \nFor testing, you must fill nulls')

# only use universe for real data simulation
X['universe'] = universe_feat(X) # N/A kernel
print('\nuniverse for simulation only \n')

# Create objects for ranges, etc
X['date_'] = X.time.dt.strftime("%Y%m%d").astype(int) 
dates = X['date_'].unique()
target = ['returnsOpenNextMktres10']
label = ['label']  
X[label[0]] = (X[target[0]] > 0).astype(int)
preproc_feats = trainFeats(X, drops=['date_', 'time', 'assetCode', 'universe', 
                                    'returnsOpenNextMktres10', 
                                    'label'])

test_pipe = preprocPipe(X, binFeat='volume')
X_p = test_pipe.fit_transform(X[preproc_feats])
X_p.head()

In [ ]:
# TRAIN THE MODEL
feats_train = X_p.columns.tolist()
print('List of training feats from the processed dataframe is created \n')

# Non training feats
feats = [col for col in X.columns if col not in preproc_feats]
print('List of non-training feats from the processed dataframe is created \n')

# Merge
X_ = pd.concat([X[feats] ,X_p],axis=1)
print('Non-training feats from unprocessed dframe and \ntraining feats from processed \
are merged. Shape: ', X_.shape)

In [ ]:
# MODEL TRAINING
train_range = range(len(dates))[:int(0.90*len(dates))]
val_range = range(len(dates))[int(0.90*len(dates)):]
print('Ranges created')

# test_range = range(len(dates))[int(0.9*len(dates)):] # N/A kernel

X_train = X_[feats_train].loc[X_['date_'].isin(dates[train_range])].values
y_train = X_[label].loc[X_['date_'].isin(dates[train_range])].values
# GETS INTERESTING -> INPUTS DATA INTO INSTANCE
lgb_train = lgb.Dataset(X_train, y_train[:,0], feature_name=feats_train)
print('Training data: ', X_train.shape, y_train.shape)


# validation data - numpys
X_val = X_[feats_train].loc[X_['date_'].isin(dates[val_range])].values
y_val = X_[label].loc[X_['date_'].isin(dates[val_range])].values
lgb_val = lgb.Dataset(X_val, y_val[:,0], feature_name=feats_train),
print('Validation data: ', X_val.shape, y_val.shape)


# test data --> N/A kernel
# X_test = X_[feats_train].loc[X_['date_'].isin(dates[test_range])].values
# y_test = X_[label].loc[X_['date_'].isin(dates[test_range])].values

# print(X_test.shape, y_test.shape)

param = {"objective" : "binary",
          "metric" : "binary_logloss",
          "verbosity" : -1,
        'random_state': 81}

# TONS OF HYPERPARAMENTERS. THIS FIT FUNC USES THE VAL SET TO TRAIN AND 
# SELECT THE BEST ITERATION

model = lgb.train(param, lgb_train, valid_sets=lgb_val, 
                  early_stopping_rounds=10) 
                  

In [ ]:
# Kernel simulation
# Define the base ahead of time
t = base.tail(1)['time'].tolist()[0]
print('Last date in the training data', t, '\n')
time = (t - dt.timedelta(days=250))
print('Past date needed for longest lag', time, '\n')

base_ = base[base.time > time]
# this could vary slightly in size

base_.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
print('Base shape: ', base_.shape)

In [ ]:
# REFRESHER WALKTHROUGH
# test = data_test.loc[test_dates[0]]
# test.reset_index(inplace=True)
# test

In [ ]:
env_sim = []
days = []
print_=False

for i in range(len(test_dates)):
    dframe = data_test.loc[test_dates[i]]
    days.append(dframe)
    

    # First day comes in...
    
    test = days[i]
    print('Day {} of {}'. format(i+1, len(test_dates)), '\n')
    if print_:
        print('Daily test shape: ', test.shape, '\n')
        print('Date: ', test.time[0], '\n')
        
    # Move assetCode from index to col
    test.reset_index(inplace=True)
    
    # Simulated predictions template with universe discrepencies
    vec = test.assetCode.tolist()
    template = pd.DataFrame({ 'assetCode' :random.sample(vec, k=int(len(vec)*.95)),
                            'confidenceValue': 0})   
    unv = template.assetCode.tolist()    
    df_unv = test[test.assetCode.isin(unv)]
    if print_:
        print('Template universe', len(unv), '\n')
        print('Simulated template unv reduction: ', len(vec) - int(len(vec)*.95), '\n')
        print('Daily test reduced for universe: ', df_unv.shape, '\n')

    # BASE
    # idenfify the oldest base date
    drop_time = base_.time.iloc[0]    
    # update the base with the test day
    base_ = pd.concat([base_, df_unv], axis=0)
    # drop the oldest date
    base_ = base_[base_.time != drop_time]
    # copy, just to be sure
    X_test = base_.copy()

    # ENGINEER FEATS
    X_test >> \
            exTractTime(col='time', 
                        atts=['weekday', 'month', 'year', 'day', 'quarter'],
                        mthds=['month_name']) >> \
            toCatFeat(feats=['year', 'quarter']) >> \
            macdFeats() >> \
            bbSqueeze() >> \
            rsiFeats()
    if print_:
        print('X_test shape after processing: ',X_test.shape, '\n')
        
    X_test = X_test.iloc[-df_unv.shape[0]:, :]
    
    if print_:
        print('Reduced: ',X_test.shape, '\n')

    # No longer able to drop nulls. Use fill instead
    X_test.drop(['sq_min'], axis=1, inplace=True)
    
    preproc_feats = trainFeats(X_test, drops=['time', 'assetCode'])
    test_pipe = preprocPipe(X_test, binFeat='volume')
    
    X_p = test_pipe.fit_transform(X_test[preproc_feats]).fillna(0)
    if print_:
        print('Transformed and nulls filled \n')

    preds = model.predict(X_p, num_iteration=model.best_iteration) * 2 - 1
    if print_:
        print('pred shape: ', preds.shape, '\n')
    
    df_pred = pd.DataFrame({'assetCode':X_test['assetCode'],'conf':preds})
    pred_dict = df_pred.set_index('assetCode').to_dict()['conf']
    template['confidenceValue'] = template.assetCode.map(pred_dict)
    # template.head()

    env_sim.append(template)
    print('\nenv appended \n')
    del template

In [ ]:
env_sim[2].head(20)

In [ ]:
# FULL TEST SET DATASET VERSION
preds = model.predict(X_test, num_iteration=model.best_iteration) 
date_vec = df['date'].loc[df['date'].isin(dates[test_range])]
u = df['universe'].loc[df['date'].isin(dates[test_range])]
actual = df[target].fillna(0).loc[df['date'].isin(dates[test_range])].values[:, 0]


custom_metric(date_vec, preds, actual, u)

In [ ]:
lgb.plot_importance(model, )

In [ ]:
importances = model.feature_importance()
importances

In [ ]:
indices = np.argsort(importances)
top = indices[::-1][:20]

In [ ]:
top

In [ ]:
importances[14] / sum(importances)

In [ ]:
importances = model.feature_importance()
indices = np.argsort(importances)
top = indices[::-1][:20]
for i in range(len(top)):
#     print(top[i])
    name = df_[feats].columns[top[i]]
    w = importances[top[i]] / sum(importances)
#     print(name)
    print('{} Feature: {}, weight: {}'.format(i+1, name, round(w, 3)))

In [ ]:
# Can probably drop the vol bins...

## Function work

In [43]:
def bbSqueeze(X):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
    Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = X.groupby('assetCode')['close']
    
    sma = asset_close.tranform(np.nansum(x[-20:]) / 20)
    std = asset_close.tranform(np.std(x[-20:]) / 20)
       
    U = sma + (std*2)    
    L = sma - (std*2) 
    
    del asset_close
    
    return (U - L)/ sma, U, D
    
   
    

## All import functions

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
import pandas_datareader as web
import gc


####################################################################    
#                  TECHNICAL INDICATOR FEATURES                    #
####################################################################




####################################################################    
#                  PIPELINE FEAUTURES                   #
####################################################################


def pipe(original):
    """
    Need to break this down 
    
    """
    class PipeInto(object):
        data = {'function': original}

        def __init__(self, *args, **kwargs):
            self.data['args'] = args
            self.data['kwargs'] = kwargs

        def __rrshift__(self, other):
            return self.data['function'](
                other, 
                *self.data['args'], 
                **self.data['kwargs']
            )

    return PipeInto



import datetime as dt

@pipe
def exTractTime(X, col, atts=list, mthds=None):

    mod = getattr(X, col)

    for i in range(len(atts)):
    
        X[atts[i]] = getattr(mod.dt, atts[i])
    
    while mthds:
        for i in range(len(mthds)):
    
            f = getattr(mod.dt, mthds[i])
            X[mthds[i]] = f()
        break
        
    return X


@pipe
def toCatFeat(X, feats=list):
    for i in feats:
        colName = i + '_obj'
        X[colName] = X[i].astype(str) + '_'
    return X



@pipe
def macdFeats(X):
    
    """
    If macd is negative/positive, the shorter term mv avg is below/above the longer 
    term and momentum is downward/upward - ish.
    
    If the signal is simply the ema of macd. If macd is less/greater than the signal,
    whether pos or neg, the momentum is bear/bullish
    
    Ex
    -----
    both signal and macd are negative. However, macd is noticably less
    negative than the signal. Interpretation: the asset has been in a short term down
    trend (short term down trend could be part of a long term down/up trend). However, 
    the momentum is bullish
    
    Metrics for indicators
    ------
    Crossover: macd is below/above signal
    
    Divergence: the price diverges by a certain threshold from macd. End of 
    current trend
    
    Dramatic Rise: macd is significantly above signal - overbought
       
    """
    
    # compute macd
    asset_close = X.groupby('assetCode')['close']
    ema_26 = asset_close.transform(lambda x: x.ewm(span=26).mean()).values
    ema_12 = asset_close.transform(lambda x: x.ewm(span=12).mean()).values
    macd = ema_12 - ema_26
    
    del asset_close, ema_12, ema_26
    
    # Need to define this feature for the groupby
    X['macd'] = macd
    sig =(
        X.groupby('assetCode')['macd'].
        transform(lambda x: x.ewm(span=9).
        mean()).
        values
        )
    X['sig'] = sig
    
    diff = macd - sig
    close = X['close'].values
    div = abs(close - macd)
    
    # Define the bools - explain thresholds
    X['macd_cross'] = np.where(macd > sig, 1, 0).astype('bool')
    X['macd_sharp_rise'] = np.where(diff > 0.09, 1, 0).astype('bool')
    X['macd_div'] = np.where(div > 75, 1, 0).astype('bool')
    
    del macd, sig, diff, close, div
    gc.collect()
    
    return X


    
@pipe
def bbSqueeze(X):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
    Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = X.groupby('assetCode')['close']
    
    sd_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            std()).
            values        
            )
    
    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values
            )
    
    # convert everything to arrays   
    close = X['close'].values   
    U = sma_20 + (sd_20*2)    
    L = sma_20 - (sd_20*2) 
    squeeze = (U - L)/ sma_20
    
    del U, L, asset_close
        
    X['squeeze'] = squeeze
    # We want the nulls - hence min period
    sq_min = (
            X.groupby('assetCode')['squeeze'].
            transform(lambda x: x.rolling(window=126, min_periods=126).
            min().
            shift())
            )
    # Use to drop null, drop from training features
    X['sq_min'] = sq_min
    
    # Define bool
    X['low_vol'] = np.where(squeeze <= sq_min, 1, 0).astype('bool')

    del squeeze, close, sq_min
    gc.collect()
        
    return X

 
    
@pipe
def rsiFeats(X):    
    """
    May consider adding bools or binning
    """
    
    def rsi(x):    
        diff = x.diff()
        mask = diff < 0
        high = abs(diff.mask(mask)).fillna(0)
        low = abs(diff.mask(~mask)).fillna(0)
        pos_rsi = high.ewm(span=14).mean()
        neg_rsi = low.ewm(span=14).mean()
        
        del diff, mask, high, low
        
        return  pos_rsi / (pos_rsi + neg_rsi)
    
    X['rsi'] = (
                X.groupby('assetCode')['close'].
                transform(lambda x: rsi(x)).
                values
                )
    
    gc.collect()
    
    return X
   


####################################################################    
#                  TRANSFORMER                           #
####################################################################

from market_trans import *
from sklearn.pipeline import make_pipeline
import sys
paths = [
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
         '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union' 
        ]

for path in paths:
    sys.path.append(path)
        
from quickpipe_mod import * 
from pandas_feature_union import *




def preprocPipe(X, binFeat):
    
    transformer_list = [ ('numeric', make_pipeline(TypeSelector(np.number),
                                QuickPipeline_mod()
                                )
                        ),('binned_features', make_pipeline(
                            TypeSelector(np.number),
                            SelectFeatures(feat_list=[binFeat]),
                            KBins(n_bins=5),
                            QuickPipeline_mod()
                             )
                        ), ('boolean_features', make_pipeline(
                                 TypeSelector(np.bool_),
                                QuickPipeline_mod(categorical_features=(TypeSelector(np.bool_).
                                                                        fit_transform(X).
                                                                        columns.
                                                                        tolist()
                                                                        ))
                                )
                       ),('categorical_features', make_pipeline(
                             TypeSelector(np.object),
                            QuickPipeline_mod() )
                            )                                           
                       ]
        
    pipe = make_pipeline(PandasFeatureUnion(transformer_list))
    print('pipeline created')
    return pipe
    





